In [9]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [10]:
from __future__ import division, print_function
# coding=utf-8
import sys
import os
import glob
import re
import numpy as np
import pandas as pd

# Keras
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer
import site

site.addsitedir('C:\Anaconda\Lib\site-packages')

In [15]:
app = Flask(__name__)
model = load_model('dl_model.h5')
model.make_predict_function()
df = pd.read_csv('dialect_dataset.csv')
def predict_model(text, model):
    MAX_NB_WORDS = 50000
    MAX_SEQUENCE_LENGTH = 250
    EMBEDDING_DIM = 100
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
    seq = tokenizer.texts_to_sequences(text)
    padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
    pred = model.predict(padded)
    labels = df['dialect'].drop_duplicates().tolist()
    print(pred, labels[np.argmax(pred)])
    return pred, labels[np.argmax(pred)]

@app.route('/')
            
def home():
    # Main page
    return render_template('home.html')

@app.route('/result', methods=['GET', 'POST'])
def result():
    if request.method == 'POST':
        # Get the text from post request
       result = request.form['Data']
       result_pred = predict_model([result],model)
       return render_template("result.html",result = result_pred)
    return None


if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Mar/2022 01:15:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2022 01:15:27] "GET /bootstrap.css HTTP/1.1" 404 -
127.0.0.1 - - [15/Mar/2022 01:15:27] "GET /style.css HTTP/1.1" 404 -
127.0.0.1 - - [15/Mar/2022 01:15:32] "POST /result HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2022 01:15:32] "GET /bootstrap.css HTTP/1.1" 404 -
127.0.0.1 - - [15/Mar/2022 01:15:32] "GET /style.css HTTP/1.1" 404 -


[[0.03603795 0.08620495 0.0523845  0.07342794 0.03555323 0.03762096
  0.05230664 0.03280691 0.06548201 0.03292128 0.04823986 0.11382581
  0.07036359 0.05971238 0.06647214 0.04794914 0.04517387 0.04351672]] EG
